In [1]:
import functools
import operator
import os
import sys

import anndata as ad
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import scanpy as sc
import scipy.stats
import seaborn as sns
import sklearn.cluster
import torch

import sklearn.linear_model
import sklearn.preprocessing
import sklearn.metrics
from matplotlib import rcParams
from networkx.algorithms.bipartite import biadjacency_matrix

import scglue
import matplotlib.pyplot as plt

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances


from utils.test_util import test_data_multiome, Siamese_Test_multiome
from utils.feature_util import feature_selection_multiome
import utils.cis_infer_util as cis_infer_util


/usr/local/lib/python3.6/dist-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
rna = ad.read_h5ad("data/rna.h5ad")
atac = ad.read_h5ad("data/atac.h5ad")

genes = scglue.genomics.Bed(rna.var.assign(name=rna.var_names).query("d_highly_variable"))
peaks = scglue.genomics.Bed(atac.var.assign(name=atac.var_names).query("d_highly_variable"))
tss = genes.strand_specific_start_site()
promoters = tss.expand(2000, 0)

pchic_graph = nx.read_graphml("data/pchic.graphml.gz")
pchic = pd.DataFrame(
    biadjacency_matrix(pchic_graph, genes.index, peaks.index, weight=None, dtype=np.float32).toarray(),
    index=genes.index, columns=peaks.index
)


In [3]:
genes = pchic.index.to_numpy()
distance = 150000
ref_tss_fn = 'utils/hg38_ref_TSS.txt'


#### Non Smoothing

In [4]:
input_mod1 = ad.read_h5ad('data/10XMultiome_GEX.PBMC.h5ad')
input_mod1.X = input_mod1.layers['counts'].copy()
input_mod2 = ad.read_h5ad('data/10XMultiome_ATAC.PBMC.h5ad')



In [5]:
#non-smoothing
cis_score_df = pd.DataFrame(np.zeros((0,6)))
cis_score_df.columns = ['genes','peaks','tss_dist','pRegion','Spearman.cor','Method']


cis_score_tmp = cis_infer_util.cis_element_score(input_mod1, input_mod2, genes, 
                                         distance, ref_tss_fn, use_rep='SiaNN',
                                         n_pseudocells=100, return_pseudo_bulk = False)
cis_score_tmp['Method'] = 'SiaNN'
cis_score_df = pd.concat([cis_score_df,cis_score_tmp])

Add suffix to cell names in RNA and ATAC data...
Clustering pseudocells...
Representation mode engaged, clustering based on combined SiaNN embedding...


/mnt/hdd/chaozhong/pylib/site-packages/scanpy/preprocessing/_normalization.py:138: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


Selecting peaks within 150000bp of the genes...
Zeros are kept and spearman correlation calculation will be done.


/home/chaozhong/.local/lib/python3.6/site-packages/scglue/num.py:152: RuntimeWarning: invalid value encountered in true_divide
  pcc = cov_mat(X, Y) / X_std.T / Y_std


In [6]:
cis_score_df['is_pcHiC'] = False
for gene in genes:
    pcHiC_bool = cis_score_df.loc[cis_score_df['genes']==gene,'peaks'].isin(pchic.columns[pchic.loc[gene,:]==1])
    cis_score_df.loc[cis_score_df['genes']==gene,'is_pcHiC'] = pcHiC_bool
    
cis_score_df.to_csv('correlation/spr.cor.pbmc.150k.NoSm.csv',index=False)
    

#### True Pair

In [7]:
sc.pp.normalize_total(input_mod1)
sc.pp.log1p(input_mod1)
sc.pp.pca(input_mod1, n_comps=30)
sc.pp.neighbors(input_mod1, n_neighbors=10)
sc.tl.umap(input_mod1)
input_mod1.X = input_mod1.layers['counts'].copy()

In [8]:
#True pair
cis_score_df = pd.DataFrame(np.zeros((0,6)))
cis_score_df.columns = ['genes','peaks','tss_dist','pRegion','Spearman.cor','Method']


cis_score_paired = cis_infer_util.cis_element_score(input_mod1, input_mod2, genes, 
                                             distance, ref_tss_fn, use_rep = 'paired',
                                             n_pseudocells=100, return_pseudo_bulk = False)
cis_score_paired['Method'] = 'True_pair'
cis_score_df = pd.concat([cis_score_df,cis_score_paired])

Add suffix to cell names in RNA and ATAC data...
Clustering pseudocells...
Paired mode engaged, clustering based on RNA dataset...


Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.
/mnt/hdd/chaozhong/pylib/site-packages/scanpy/preprocessing/_normalization.py:138: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


Selecting peaks within 150000bp of the genes...
Zeros are kept and spearman correlation calculation will be done.


/home/chaozhong/.local/lib/python3.6/site-packages/scglue/num.py:152: RuntimeWarning: invalid value encountered in true_divide
  pcc = cov_mat(X, Y) / X_std.T / Y_std


In [9]:
cis_score_df['is_pcHiC'] = False
for gene in genes:
    pcHiC_bool = cis_score_df.loc[cis_score_df['genes']==gene,'peaks'].isin(pchic.columns[pchic.loc[gene,:]==1])
    cis_score_df.loc[cis_score_df['genes']==gene,'is_pcHiC'] = pcHiC_bool
    
cis_score_df.to_csv('correlation/spr.cor.pbmc.150k.Pair.csv',index=False)
    

#### Smoothing

In [7]:
num_nn = 20
input_mod1_sm = ad.read_h5ad('data/10XMultiome_GEX.PBMC.smoothed.%sNN.h5ad'%(num_nn))
input_mod1_sm.X = input_mod1_sm.layers['smoothed_counts'].copy()

input_mod2_sm = ad.read_h5ad('data/10XMultiome_ATAC.PBMC.smoothed.%sNN.h5ad'%(num_nn))
input_mod2_sm.X = input_mod2_sm.layers['smoothed_counts'].copy()


In [8]:
#Smothing
cis_score_df = pd.DataFrame(np.zeros((0,6)))
cis_score_df.columns = ['genes','peaks','tss_dist','pRegion','Spearman.cor','Method']


cis_score_tmp = cis_infer_util.cis_element_score(input_mod1_sm, input_mod2_sm, genes, 
                                         distance, ref_tss_fn, use_rep='SiaNN',
                                         n_pseudocells=100, return_pseudo_bulk = False)
cis_score_tmp['Method'] = '%sNN'%(num_nn)
cis_score_df = pd.concat([cis_score_df,cis_score_tmp])

Add suffix to cell names in RNA and ATAC data...
Clustering pseudocells...
Representation mode engaged, clustering based on combined SiaNN embedding...


/mnt/hdd/chaozhong/pylib/site-packages/scanpy/preprocessing/_normalization.py:138: UserWarning: Revieved a view of an AnnData. Making a copy.
  view_to_actual(adata)


Selecting peaks within 150000bp of the genes...
Zeros are kept and spearman correlation calculation will be done.


/home/chaozhong/.local/lib/python3.6/site-packages/scglue/num.py:152: RuntimeWarning: invalid value encountered in true_divide
  pcc = cov_mat(X, Y) / X_std.T / Y_std


In [9]:
cis_score_df['is_pcHiC'] = False
for gene in genes:
    pcHiC_bool = cis_score_df.loc[cis_score_df['genes']==gene,'peaks'].isin(pchic.columns[pchic.loc[gene,:]==1])
    cis_score_df.loc[cis_score_df['genes']==gene,'is_pcHiC'] = pcHiC_bool
    
cis_score_df.to_csv('correlation/spr.cor.pbmc.150k.%sNN.csv'%(num_nn),index=False)
    